In [5]:
# Equation of the dynamics.

# Wheel radius, mass, inertia
r = var('r')
mw = var('mw', latex_name='m_w')
J = var('J')

# Pendulum mass, length, inertia about CoM
m = var('m')
l = var('l')
I = var('I')

# Gravity
g = var('g')
        
# State variables
x = var('x')
dx = var('dx',  latex_name='\\dot{x}')
ddx = var('ddx',  latex_name='\\ddot{x}')
theta = var('theta', latex_name='\\theta')
dtheta = var('dtheta',  latex_name='\\dot{\\theta}')
ddtheta = var('ddtheta',  latex_name='\\ddot{\\theta}')
phi = var('phi', latex_name='\\phi')
dphi = var('dphi',  latex_name='\\dot{\\phi}')
ddphi = var('ddphi',  latex_name='\\ddot{\\phi}')

q = [x, theta, phi]
dq = [dx, dtheta, dphi]
ddq = [ddx, ddtheta, ddphi]

# Compute kinetic energy
K = 1 / 2 * ( mw * dx**2 + J * (dtheta + dphi)**2 + I * dtheta**2 + m * (dx**2 + 2 * l * dx * dtheta * cos(theta) + (l * dtheta)**2))
# Compute potential energy
E = m * g * l * cos(theta)
# Lagrangian
L = K - E

# Write Lagrange equation
dL_dq = matrix([diff(L, i) for i in q]).T
dL_ddq = matrix([diff(L, i) for i in dq]).T
D_Dt_dL_ddq =  matrix([sum([diff(dL_ddq[i], q[j]) * dq[j] + diff(dL_ddq[i], dq[j]) * ddq[j] for j in range(len(q))]) for i in range(len(q))])

lagrange_equations = (D_Dt_dL_ddq - dL_dq).simplify_full()

print("Lagrange equation:")
print(lagrange_equations)

# Extract mass matrix from this equation
# Given an expression and variable, extract coefficient of this variable in the expression
def get_coeff(expr, var):
    coeffs = expr.coefficients(var)
    if len(coeffs) < 2:
        return 0
    return coeffs[1][0]

H = matrix([[get_coeff(lagrange_equations[j][0], i) for i in ddq] for j in range(len(q))])
H = H.simplify_full()
print("Mass matrix:")
print(H)

# Compute non-linear effects such that the lagrange equation (without external forces) writes M ddq + nle = 0
nle = (lagrange_equations - H * matrix(ddq).T).simplify_full()
print("Nonlinear effects:")
print(nle)

Lagrange equation:
[               -dtheta^2*l*m*sin(theta) + ddtheta*l*m*cos(theta) + ddx*m + ddx*mw]
[ddtheta*l^2*m + ddx*l*m*cos(theta) - g*l*m*sin(theta) + J*ddphi + (I + J)*ddtheta]
[                                                              J*ddphi + J*ddtheta]
Mass matrix:
[        m + mw l*m*cos(theta)              0]
[l*m*cos(theta)  l^2*m + I + J              J]
[             0              J              J]
Nonlinear effects:
[-dtheta^2*l*m*sin(theta)]
[       -g*l*m*sin(theta)]
[                       0]


In [7]:
# Now lets add the wheel constraint.
# We have dx = -r (dtheta + dphi), thus calling lamda the corresponding force: lambda is equal to the first lagrange
# equation, and we add r lamda to each of the other equation

lamda_r = lagrange_equations[0,0] * r
lagrange_constraint = matrix([lagrange_equations[1,0] + lamda_r, lagrange_equations[2,0] + lamda_r]).T
# Now run the substitution: dx = r (dtheta + dphi), ddx = r (ddtheta + ddphi)
lagrange_constraint = lagrange_constraint.substitute(dx=r * (dtheta + dphi))
lagrange_constraint = lagrange_constraint.substitute(ddx=r * (ddtheta + ddphi))
lagrange_constraint = lagrange_constraint.simplify_full()

# Finally, extract H and nle like before.
print("Constraint equation:")
print(lagrange_constraint)

ddq_c = [ddtheta, ddphi]
print("\nConstraint mass:")
H = matrix([[get_coeff(lagrange_constraint[j][0], i) for i in ddq_c] for j in range(len(ddq_c))])
H = H.simplify_full()
print(H)

nle = (lagrange_constraint - H * matrix(ddq_c).T).simplify_full()
print("\nConstraint nonlinear effects:")
print(nle)


Constraint equation:
[(ddphi + 2*ddtheta)*l*m*r*cos(theta) + ddtheta*l^2*m + ((ddphi + ddtheta)*m + (ddphi + ddtheta)*mw)*r^2 + J*ddphi + (I + J)*ddtheta - (dtheta^2*l*m*r + g*l*m)*sin(theta)]
[                                           -dtheta^2*l*m*r*sin(theta) + ddtheta*l*m*r*cos(theta) + ((ddphi + ddtheta)*m + (ddphi + ddtheta)*mw)*r^2 + J*ddphi + J*ddtheta]

Constraint mass:
[2*l*m*r*cos(theta) + l^2*m + (m + mw)*r^2 + I + J               l*m*r*cos(theta) + (m + mw)*r^2 + J]
[              l*m*r*cos(theta) + (m + mw)*r^2 + J                                  (m + mw)*r^2 + J]

Constraint nonlinear effects:
[-(dtheta^2*l*m*r + g*l*m)*sin(theta)]
[          -dtheta^2*l*m*r*sin(theta)]


In [61]:
print(latex(H))
print()
print(latex(nle))

\left(\begin{array}{rr}
2 \, l m r \cos\left({\theta}\right) + l^{2} m + {\left(m + {m_w}\right)} r^{2} + I + J & l m r \cos\left({\theta}\right) + {\left(m + {m_w}\right)} r^{2} + J \\
l m r \cos\left({\theta}\right) + {\left(m + {m_w}\right)} r^{2} + J & {\left(m + {m_w}\right)} r^{2} + J
\end{array}\right)

\left(\begin{array}{r}
-{\left({\dot{\theta}}^{2} l m r + g l m\right)} \sin\left({\theta}\right) \\
-{\dot{\theta}}^{2} l m r \sin\left({\theta}\right)
\end{array}\right)


In [69]:
# Compute inverse of H_psi
H_psi = matrix([[m * l**2 + I, m * l * r], [m * l * r, (m + mw)*r**2 + J]])
H_inv = H_psi.inverse().simplify_full()

# Compute coefficients of A
A_coeffs = (H_inv * matrix([m * l * g, 0]).T).simplify_full()
print(A_coeffs )
B_coeffs = (H_inv * matrix([-1, 1]).T).simplify_full()
print(B_coeffs )


[(J*g*l*m + (g*l*m^2 + g*l*m*mw)*r^2)/(J*l^2*m + (I*m + (l^2*m + I)*mw)*r^2 + I*J)]
[                        -g*l^2*m^2*r/(J*l^2*m + (I*m + (l^2*m + I)*mw)*r^2 + I*J)]
[-(l*m*r + (m + mw)*r^2 + J)/(J*l^2*m + (I*m + (l^2*m + I)*mw)*r^2 + I*J)]
[        (l^2*m + l*m*r + I)/(J*l^2*m + (I*m + (l^2*m + I)*mw)*r^2 + I*J)]
